In [9]:
import pandas as pd
crime_train = pd.read_csv('train.csv', na_values='?')
crime_test = pd.read_csv('test.csv', na_values='?')
print crime_train.shape
print crime_test.shape
print crime_train.isnull().sum()
print crime_test.isnull().sum()

(878049, 9)
(884262, 7)
Dates         0
Category      0
Descript      0
DayOfWeek     0
PdDistrict    0
Resolution    0
Address       0
X             0
Y             0
dtype: int64
Id            0
Dates         0
DayOfWeek     0
PdDistrict    0
Address       0
X             0
Y             0
dtype: int64
